In [71]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
import pickle
import re 

#from petdata_functions import CleanPetData
#from petdata_functions import ReformatDates


#read in data 
chidat = pd.read_csv("Chicago_adopted_20k_Jan_21.csv") 
chidat['City'] = 'Chicago'

houstdat = pd.read_csv("Houston_adopted_20k_Jan_21.csv") 
houstdat['City'] = 'Houston'

elpdat = pd.read_csv("ElPaso_adopted_20k_Jan_21.csv") 
elpdat['City'] = 'ElPaso'

Minndat = pd.read_csv("Chicago_adopted_20k_Jan_21.csv") 
Minndat['City'] = 'Minne'

dendat = pd.read_csv("Denver_adopted_20k_Jan_21.csv") 
dendat['City'] = 'Denver'

stldat = pd.read_csv("StL_adopted_20k_Jan_21.csv")
stldat['City'] = 'StLouis'

inddat = pd.read_csv("Indy_adopted_20k_Jan_21.csv")
inddat['City'] = 'Indy'

petdat = pd.concat([chidat,houstdat,elpdat,Minndat,dendat, stldat, inddat])

#print(petdat.iloc[0]['photos'])
#print(list(petdat.columns))


#reformat dates and calculate difference in dates (= target variable)
fmt ='%Y-%m-%dT%H:%M:%S+0000'
petdat['published_at'] =  pd.to_datetime(petdat['published_at'], format=fmt)
petdat['status_changed_at'] =  pd.to_datetime(petdat['status_changed_at'], format=fmt)
petdat['time_diff'] = petdat['status_changed_at'] - petdat['published_at']
petdat['target_hour'] = petdat.time_diff.astype('timedelta64[h]')
petdat['log_target_hour'] = np.log(petdat['target_hour'])

#calculate number of tags
petdat['num_tags']= petdat['tags'].str.count(',', re.I) 

#calculate number of photos
petdat['num_photos']= petdat['photos'].str.count('small', re.I) 

#calculate number of words in description 
petdat['num_words_desc']= petdat['description'].str.count(' ', re.I) 

#remove outliers from logs to avoid -inf/NaN
q_low = petdat["log_target_hour"].quantile(0.02)
q_hi  = petdat["log_target_hour"].quantile(0.80)
petdat = petdat[(petdat["log_target_hour"] < q_hi) & (petdat["log_target_hour"] > q_low)]

print(petdat.head())

/opt/anaconda3/envs/insight/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/anaconda3/envs/insight/lib/python3.6/site-packages/pandas/core/series.py:856: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


   Unnamed: 0        id organization_id  \
1           1  47171077           IL496   
6           6  47152094           IN111   
7           7  47150453           IL663   
8           8  47149430           IL145   
9           9  47148767           IL782   

                                                 url type species    age  \
1  https://www.petfinder.com/dog/keeper-47171077/...  Dog     Dog  Adult   
6  https://www.petfinder.com/dog/tiptoe-47152094/...  Dog     Dog  Young   
7  https://www.petfinder.com/dog/lola-47150453/il...  Dog     Dog   Baby   
8  https://www.petfinder.com/dog/max-47149430/il/...  Dog     Dog   Baby   
9  https://www.petfinder.com/dog/mama-mia-4714876...  Dog     Dog  Adult   

   gender    size    coat  ...      _links.self.href _links.type.href  \
1    Male  Medium  Medium  ...  /v2/animals/47171077    /v2/types/dog   
6    Male   Large   Short  ...  /v2/animals/47152094    /v2/types/dog   
7  Female  Medium   Short  ...  /v2/animals/47150453    /v2/types

In [72]:
traindat = petdat[['age',
                   'organization_id',
                   'contact.address.city',
                   'City',
                   'size',
                   'coat',
                   'distance',
                   'colors.primary',
                   'colors.secondary',
                   'breeds.primary',
                   'breeds.secondary',
                   'breeds.mixed',
                   'breeds.unknown',
                   'attributes.spayed_neutered',
                   'attributes.house_trained',
                   'attributes.special_needs',
                   'attributes.shots_current',
                   'environment.cats',
                   #'environment.children',
                   'environment.dogs',
                   'log_target_hour']]


#get rid of extreme outliers in target var to prevent model problems
#q_low = traindat["log_target_hour"].quantile(0.01)
#q_hi  = traindat["log_target_hour"].quantile(0.99)
#traindat = traindat[(traindat["log_target_hour"] < q_hi) & (traindat["log_target_hour"] > q_low)]

#make a df 'X' of features to be used  as predictors in the model, right now all of traindat

X = traindat.drop(columns = ['log_target_hour'])

#one hot coding for features 
cols_to_transform = ['age',
                     'organization_id',
                     'contact.address.city',
                     'City',
                     'size',
                     'coat',
                     'colors.primary',
                     'colors.secondary',
                     'breeds.primary',
                     'breeds.secondary',
                     'breeds.mixed',
                     'breeds.unknown',
                     'attributes.spayed_neutered',
                     'attributes.house_trained',
                     'attributes.special_needs',
                     'attributes.shots_current',
                     'environment.cats',
                     #'environment.children',
                     'environment.dogs']
X = pd.get_dummies(X, columns = cols_to_transform)

#fit lm
lm = LinearRegression()

lm.fit(X, traindat.log_target_hour)
print(lm.score(X, traindat.log_target_hour))
print('Coefficients: \n', lm.coef_)
pd.DataFrame({'features': X.columns, 'estimatedCoefficients': lm.coef_})[['features', 'estimatedCoefficients']]


0.2255903449555938
Coefficients: 
 [ 5.36329256e-02 -2.07000420e+09 -2.07000420e+09 ... -4.84277010e-02
  1.90839259e-01  3.27790380e-02]


,features,estimatedCoefficients
0,distance,5.363293e-02
1,age_Adult,-2.070004e+09
2,age_Baby,-2.070004e+09
3,age_Senior,-2.070004e+09
4,age_Young,-2.070004e+09
...,...,...
1404,attributes.shots_current_True,-8.805851e+08
1405,environment.cats_False,1.897804e-01
1406,environment.cats_True,-4.842770e-02
1407,environment.dogs_False,1.908393e-01
